### 8.1. Changing the String Representation of Instances

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

In [3]:
p = Pair(3,4)
print(p)
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))

(3, 4)
p is Pair(3, 4)
p is (3, 4)


### 8.2. Customizing String Formatting

In [3]:
_formats = {'ymd':'{d.year}-{d.month}-{d.day}',
           'mdy':'{d.month}/{d.day}/{d.year}'}
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [4]:
d = Date(2012, 12, 23)
format(d, 'mdy')

'12/23/2012'

### 8.3. Making Objects Support the Context-Management
Protocol
define __init__ __enter__ __exit__ function on the python class, the __exit__ function will do the clean job after __enter__

In [13]:
from socket import socket, AF_INET, SOCK_STREAM
class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        print('init funcitno to do the initializing')
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None
    def __enter__(self):
        print('entering the enter funciton')
        if self.sock is not None:
            raise RuntimeError("Already connected")
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    def __exit__(self, exc_ty, exc_val, tb):
        print('exit function to destory connection')
        self.sock.close()
        self.sock = None
from functools import partial

conn = LazyConnection(('www.python.org', 80))
with conn as s:
    print('enter connection')
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 1892), b''))
    print(resp)
# test code for create another connection
conn2 = LazyConnection(('www.python2.org', 80))
with conn2 as s:
    print('enter connection')
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 1892), b''))
    print(resp)

init funcitno to do the initializing
entering the enter funciton
enter connection
b'HTTP/1.1 301 Moved Permanently\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nContent-Length: 0\r\nAccept-Ranges: bytes\r\nDate: Mon, 26 Mar 2018 03:44:38 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\nX-Served-By: cache-hnd18740-HND\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1522035878.384122,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains\r\n\r\n'
exit function to destory connection
init funcitno to do the initializing
entering the enter funciton
enter connection
b"HTTP/1.1 200 OK\r\nDate: Mon, 26 Mar 2018 3:44:34 GMT\r\nConnection:Keep-Alive\r\nContent-Length: 606\r\nCache-Control: private, no-cache, no-store, max-age=0\r\nExpires: Mon, 01 Jan 1990 0:00:00 GMT\r\n\r\n<html><head><title>Not Found</title></head><body><center><br><br><br><br><br><b><big>Not Found</big></b><br><br><table cellpadding=10 cellspacing=0 width=400 border=

### 8.4. Saving Memory When Creating a Large Number of
Instances

In [26]:
class Date:
    __slots__ = ['year', 'month', 'day','week']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
p = Date(2012, 2, 3)
p.month = 3
print(p.month)

3


### 8.5. Encapsulating Names in a Class
Python 用下划线作为变量前缀和后缀指定特殊变量。


_xxx 不能用'from moduleimport *'导入

__xxx__ 系统定义名字

__xxx 类中的私有变量名


核心风格：避免用下划线作为变量名的开始。
 

结论：

1、_xxx 不能用于’from module import *’ 以单下划线开头的表示的是protected类型的变量。

即保护类型只能允许其本身与子类进行访问。

2、__xxx 双下划线的表示的是私有类型的变量。只能是允许这个类本身进行访问了。连子类也

不可以

3、__xxx___ 定义的是特列方法。像__init__之类的

In [38]:
class Temple:
    __year = 1
    def changeYear(self, year):
        self.__year = year
    def printyear(self):
        print(self.__year)
t = Temple()
t.changeYear(10)
t.printyear()
t.__year = 11
t.printyear()
t.changeYear(12)
t.printyear()

10
10
12


### 8.6. Creating Managed Attributes
Problem
You want to add extra processing (e.g., type checking or validation) to the getting or
setting of an instance attribute

In [46]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
    @property
    def first_name(self):
        print('call the getter')
        return self._first_name
    @first_name.setter
    def first_name(self, value):
        print('call the setter')
        if not isinstance(value, str):
            raise TypeError('Expectd a string')
        self._first_name = value
    @first_name.deleter
    def first_name(self):
        print('call the deleter')
        raise AttributeError("can't delete attribute")
a = Person('Guido')
a.first_name
a.first_name  = '100'

call the setter
call the getter
call the setter


### 8.7. Calling a Method on a Parent Class

In [51]:
class A:
    def spam(self):
        print('a.spam')
class B(A):
    def spam(self):
        print('B.spam')
        super().spam()
b = B()
b.spam()

B.spam
a.spam


In [67]:
class Base:
    def __init__(self):
        print('Base.__init__')
class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')
class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
class C(A,B):
    def __init__(self):
        super().__init__()
        print('C.__init__')
c = C()
C.__mro__

Base.__init__
B.__init__
A.__init__
C.__init__


(__main__.C, __main__.A, __main__.B, __main__.Base, object)

#### smaple of MRO

In [65]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()
class B:
    def spam(self):
        print('B.spam')
class C(A, B):
    pass
c = C()
c.spam()
C.__mro__

A.spam
B.spam


(__main__.C, __main__.A, __main__.B, object)

In [71]:
class Init(object):
    def __init__(self, value):
        print('Init')
        self.value = value
        print(self.value)


class Add2(Init):
    def __init__(self, value):
        super(Add2, self).__init__(value)
        print('Add2')
        # self.value += 2
        self.value = 2 + self.value
        print(self.value)


class Mul5(Init):
    def __init__(self, value):
        super(Mul5, self).__init__(value)
        print('Mul5')
        # self.value *= 5
        self.value = self.value * 5
        print(self.value)


class Pro(Mul5, Add2):
    pass


class Incr(Pro):
    def __init__(self, value):
        super(Incr, self).__init__(value)
        # self.value += 1
        self.value = self.value + 1
        print(self.value)

p = Incr(5)
print(p.value) #查看你的结果如果是36 
Incr.__mro__

Init
5
Add2
7
Mul5
35
36
36


(__main__.Incr,
 __main__.Pro,
 __main__.Mul5,
 __main__.Add2,
 __main__.Init,
 object)

### 8.8. Extending a Property in a Subclass

In [84]:
class Person:
    def __init__(self, name):
        self.name = name
    @property
    def name(self):
        return self._name
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")
class SubPerson(Person):
    @property
    def name(self):
        print("Getting name")
        return super().name
    @name.setter
    def name(self, value):
        print("setting name to ", value)
        super(SubPerson, SubPerson).name.__set__(self, value)
    @name.deleter
    def name(self):
        print("deleting name")
        super(SubPerson, SubPerson).name.__delete__(self)
s = SubPerson("Guido")
print(s.name)
s.name = "Larry"
s.name
print(s.name)

setting name to  Guido
Getting name
Guido
setting name to  Larry
Getting name
Getting name
Larry
